In [1]:
!pip install pycryptodomex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.5 MB/s eta 0:00:00


In [9]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms
from cryptography.hazmat.primitives.ciphers.aead import ChaCha20Poly1305
from cryptography.hazmat.backends import default_backend
from pathlib import Path
from Cryptodome.Cipher import AES
from Cryptodome.Random import get_random_bytes
from Cryptodome.Util.Padding import pad, unpad
from time import time
import random
import hashlib

In [10]:
from google.colab import files
import os

In [4]:
book=files.upload

In [35]:
start_time1 = time()
book = Path("lorem10.txt").read_text().encode("utf-8")
print(book)
encryption_time1 = time() - start_time1
print("Tiempo de lectura del archivo: {} segundos".format(encryption_time1))

b'Lorem ipsum dolor sit amet consectetur adipisicing elit. Fugit, aliquid?'
Tiempo de lectura del archivo: 0.002010822296142578 segundos


# Algoritmo Simétrico: Chacha 20 

Generación de claves

In [37]:
start_time2 = time()
key = get_random_bytes(32)  # Clave de 256 bits (32 bytes)
nonce = get_random_bytes(16)
print("Clave: ", key,nonce)
encryption_time2 = time() - start_time2
print("Tiempo de generación de clave: {} segundos".format(encryption_time2))

Clave:  b'/l\xbaO_\xb2\xb49U\x19\xdc\xf9\x05\x1a\xc1\xa8\xe6\xe3q\xd8\x03@g\xea\n\t\xbc\xb4l5P\x06' b',\xabU\xf1N\xf61\xaf\xe10\x177`\xc2Ta'
Tiempo de generación de clave: 0.0002522468566894531 segundos


Cifrado

In [38]:
start_time3 = time()
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)
encryptor = cipher.encryptor()
ciphertext = encryptor.update(book) + encryptor.finalize()
print("Clave: ",ciphertext.hex())
encryption_time3 = time() - start_time3
print("Tiempo de cifrado: {} segundos".format(encryption_time3))

Clave:  3ed70d961397b07a159f66d6904a2bb3b732fd8561fc392e519c19d0e63af84232195538cad8abc6409b9606bae074e10b0ad36f289e6dc8889c9b89f7b0af5343d31f34901d3080
Tiempo de cifrado: 0.0024995803833007812 segundos


Descifrado

In [39]:
start_time4 = time()
cipher = Cipher(algorithms.ChaCha20(key, nonce), mode=None)
decryptor = cipher.decryptor()
plaintext = decryptor.update(ciphertext) + decryptor.finalize()
print("Clave: ", plaintext.decode())
encryption_time4 = time() - start_time4
print("Tiempo de descifrado: {} segundos".format(encryption_time4))

Clave:  Lorem ipsum dolor sit amet consectetur adipisicing elit. Fugit, aliquid?
Tiempo de descifrado: 0.0015988349914550781 segundos


# Algoritmo Simétrico: AES

Generación de claves

In [41]:
start_time1 = time()
key = get_random_bytes(16)  # Clave de 16 bytes (128 bits)
iv = get_random_bytes(AES.block_size)
print("Clave: ", key,iv)
encryption_time1 = time() - start_time1
print("Tiempo de generación de clave: {} segundos".format(encryption_time1))

Clave:  b'x\n\xbd\xdc\x18\n:<z\x16:\xb2j\xc4\xc0R' b'<-\x94\xe6\x17l `+\xdf1\xca\xed\xc0C3'
Tiempo de generación de clave: 0.0022780895233154297 segundos


Cifrado

In [42]:
start_time1 = time()
cipher = AES.new(key, AES.MODE_CBC, iv)
ciphertext = cipher.encrypt(pad(book, AES.block_size))
print("Clave: ", book.hex())
encryption_time1 = time() - start_time1
print("Tiempo de cifrado del archivo: {} segundos".format(encryption_time1))

Clave:  4c6f72656d20697073756d20646f6c6f722073697420616d657420636f6e7365637465747572206164697069736963696e6720656c69742e2046756769742c20616c69717569643f
Tiempo de cifrado del archivo: 0.0019719600677490234 segundos


Descifrado

In [43]:
start_time1 = time()
cipher = AES.new(key, AES.MODE_CBC, iv)
book = unpad(cipher.decrypt(ciphertext), AES.block_size)
print("Clave: ", book.decode())
encryption_time1 = time() - start_time1
print("Tiempo de descifrado del archivo: {} segundos".format(encryption_time1))

Clave:  Lorem ipsum dolor sit amet consectetur adipisicing elit. Fugit, aliquid?
Tiempo de descifrado del archivo: 0.0004916191101074219 segundos


# Algoritmo Asimétrico: Diffie Hellman

Generación de clave

In [44]:
def power_modulo(base, exponent, modulus):
    result = 1
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        base = (base * base) % modulus
        exponent //= 2
    return result
prime = 98764321261
generator = 5
def generate_keys(prime, generator):
    private_key = random.randint(2, prime - 2)
    public_key = power_modulo(generator, private_key, prime)
    return (private_key, public_key)

def generate_shared_secret(private_key, public_key, prime):
    shared_secret = power_modulo(public_key, private_key, prime)
    return shared_secret

start_time5 = time()
alice_private_key, alice_public_key = generate_keys(prime, generator)
print("Clave publica Alice",alice_public_key)
print("Clave privada Alice",alice_private_key)
# Generar las claves de Bob
bob_private_key, bob_public_key = generate_keys(prime, generator)
print("Clave publica Bob",bob_public_key)
print("Clave privada Bob",bob_private_key)
encryption_time5 = time() - start_time5
print("Tiempo de generación de clave: {} segundos".format(encryption_time5))

Clave publica Alice 31322062936
Clave privada Alice 76620162131
Clave publica Bob 54562693724
Clave privada Bob 6139223495
Tiempo de generación de clave: 0.006472349166870117 segundos


# Función Hash: SHA_512

Generación de clave

In [45]:
start_time1 = time()
hash_object = hashlib.sha512()
print("Clave: ", hash_object.hexdigest())
encryption_time1 = time() - start_time1
print("Tiempo de generación de clave: {} segundos".format(encryption_time1))

Clave:  cf83e1357eefb8bdf1542850d66d8007d620e4050b5715dc83f4a921d36ce9ce47d0d13c5d85f2b0ff8318d2877eec2f63b931bd47417a81a538327af927da3e
Tiempo de generación de clave: 0.001970052719116211 segundos


Cifrado

In [46]:
def sha512_hash(message):
    hash_object = hashlib.sha512()
    hash_object.update(message)
    return hash_object.hexdigest()

start_time6 = time()
sha512_result = sha512_hash(book)
print("Mensaje:", book)
print("Hash SHA-512:", sha512_result)
encryption_time6 = time() - start_time6
print("Tiempo de cifrado: {} segundos".format(encryption_time6))

Mensaje: b'Lorem ipsum dolor sit amet consectetur adipisicing elit. Fugit, aliquid?'
Hash SHA-512: eb864f3607eb9ce0510b32b0f25dbc15f01b34917f5d89f811e76767281ec20fb70258bb76b6a21021b16963baf9f32982354dcc2da77b58c6f0ca1a439d3931
Tiempo de cifrado: 0.0027320384979248047 segundos
